In [30]:
import gc  
import os  
import time  
import warnings 
from itertools import combinations  
from warnings import simplefilter 
import joblib  
import playground.optivarfuncs as of
import lightgbm as lgb  
import numpy as np  
import pandas as pd  
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import KFold, TimeSeriesSplit  
warnings.filterwarnings("ignore")
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
is_offline = False 
is_train = True  
is_infer = True 
max_lookback = np.nan 
split_day = 435  

# Settings
There are 480 dates, 5 days a week or 96 weeks

In [31]:
class CONFIG:    
    #take last 3 months worth? or roughly 12*5=60.  So we want from (480-60) to 480
    # start_date=420

    #take last 1 months worth? or roughly 4*5=20.  So we want from (480-20) to 480
    start_date=460
    

    use_subset_of_data=False

In [32]:
#logging
import logging
# set up logging to file - see previous section for more details
logging.basicConfig(level=logging.DEBUG,
                    filename='logg.log',
                    filemode='w')
# define a Handler which writes INFO messages or higher to the sys.stderr
console = logging.StreamHandler()
# add the handler to the root logger
logging.getLogger().addHandler(console)
logger=logging.getLogger()

#use following to enable and disable
logger.disabled = True
# logger.disabled = False


In [33]:
# Tracking kernel memory usage:-  
from os import path, walk, getpid;
from psutil import Process;
def GetMemUsage():
    """
    This function defines the memory usage across the kernel. 
    Source-
    https://stackoverflow.com/questions/61366458/how-to-find-memory-usage-of-kaggle-notebook
    """;
    
    pid = getpid();
    py = Process(pid);
    memory_use = py.memory_info()[0] / 2. ** 30;
    return f"RAM usage = {memory_use :.4} GB";
GetMemUsage()

'RAM usage = 0.4286 GB'

In [34]:
from gc import collect;
collect()

5

# Data Loading and Preprocessing 




In [35]:
# df = pd.read_csv("/kaggle/input/optiver-trading-at-the-close/train.csv")
df = pd.read_csv("./data/train.csv")

# Dropping rows with null targets:-
# drop_idx = df.loc[df['target'].isna(), 'target'].index.to_list();
# df    = df.drop(drop_idx, axis=0);

df = df.dropna(subset=["target"])  #drop all rows with NaN in target
df.reset_index(drop=True, inplace=True)
df_shape = df.shape
df.drop(columns=['row_id'],inplace=True)

In [36]:
# !pip install seaborn

# import seaborn as sns
# import matplotlib.pyplot as plt;
# # logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
# fig, ax = plt.subplots(1,1, figsize = (5, 8));
# sns.heatmap(df.describe().loc[['min', 'max']].T, cbar = False, 
#             cmap = 'icefire', annot = True,
#             linewidths = 2.5, linecolor = 'white', fmt = ',.4f', 
#             annot_kws= {'fontweight': 'bold', 'fontsize': 10.5,},
#             ax = ax
#            );
# ax.set_title(f"\nMin-max across columns in the train data\n");
# plt.show();

In [37]:
if(CONFIG.use_subset_of_data):
    #just take the last 4 weeks
    df=df[df.date_id>CONFIG.start_date]
print(f'df shape={df.shape}')

df shape=(5237892, 16)


# Memory Optimization

In [38]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
               
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    if verbose:
        logger.info(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        logger.info(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        logger.info(f"Decreased by {decrease:.2f}%")
    return df


In [39]:
%%time 

def MakeFtre(df : pd.DataFrame, prices: list, median_vol: pd.DataFrame) -> pd.DataFrame:
    """
    This function creates new features using the price columns. This was used in a baseline notebook as below-
    https://www.kaggle.com/code/yuanzhezhou/baseline-lgb-xgb-and-catboost
    
    Inputs-
    df:- pd.DataFrame -- input dataframe
    cols:- price columns for transformation
    median_vol:- pd.DataFrame -- stock wise median volume for train set
    
    Returns-
    df:- pd.DataFrame -- dataframe with extra columns
    """;
    
    features = ['overall_medvol', "first5min_medvol", "last5min_medvol",
                'seconds_in_bucket', 'imbalance_buy_sell_flag',
                'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2' 
               ];
    
    df = df.merge(median_vol, how= 'left', left_on = "stock_id", right_index = True);
    
    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)').astype(np.float32);
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)').astype(np.float32);
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                df[f'{a}_{b}_imb'] = df.eval(f'({a}-{b})/({a}+{b})');
                features.append(f'{a}_{b}_imb'); 
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = df[[a,b,c]].max(axis=1);
                    min_ = df[[a,b,c]].min(axis=1);
                    mid_ = df[[a,b,c]].sum(axis=1)-min_-max_;

                    df[f'{a}_{b}_{c}_imb2'] = ((max_-mid_)/(mid_-min_)).astype(np.float32);
                    features.append(f'{a}_{b}_{c}_imb2');
    
    return df[features];

     

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


## Deal with NaNs in near_price and far_price

In [40]:
collect()
GetMemUsage()

'RAM usage = 1.612 GB'

In [41]:
dfgb = df.groupby(['stock_id','date_id']).median()
# dfgb = None

In [42]:
%%time
bs=of.bfs(['near_price','far_price'],df,dfgb)
df = df.apply(bs.backfill, axis=1)

CPU times: user 15min 19s, sys: 4.16 s, total: 15min 23s
Wall time: 15min 23s


In [29]:
%%time
bs=of.bfs(['near_price','far_price'],df)
df = df.apply(bs.backfill, axis=1)

CPU times: user 18.6 s, sys: 131 ms, total: 18.7 s
Wall time: 18.7 s


In [14]:
# %%time
# # dfc=df.copy(deep=True)
# # dfgb = df.groupby(['stock_id','date_id']).median()
# dfgb = None
# bs=of.bfs(['near_price','far_price'],df)
# df = df.apply(bs.backfill, axis=1)

CPU times: user 7min 17s, sys: 3.59 s, total: 7min 20s
Wall time: 7min 20s


In [15]:
# %debug

In [43]:
df.isnull().sum()

stock_id                     0
date_id                      0
seconds_in_bucket            0
imbalance_size             132
imbalance_buy_sell_flag      0
reference_price            132
matched_size               132
far_price                    0
near_price                   0
bid_price                  132
bid_size                     0
ask_price                  132
ask_size                     0
wap                        132
target                       0
time_id                      0
syn_near_price               0
syn_far_price                0
dtype: int64

In [44]:
#check to see if the correct value is propegating forward
df[((df.stock_id==1) & (df.date_id<=462))].tail()
# df[((df.stock_id==1) & (df.date_id==423))].head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,syn_near_price,syn_far_price
5038893,1.0,462.0,500.0,967601.00,-1.0,0.996414,10882674.73,0.995459,0.995811,0.996313,594.96,0.996614,25591.02,0.996320,0.979900,25460.0,0.0,0.0
5039093,1.0,462.0,510.0,930576.97,-1.0,0.996213,10899793.36,0.995459,0.995811,0.995961,396.50,0.996464,2975.25,0.996021,10.980368,25461.0,0.0,0.0
5039293,1.0,462.0,520.0,914652.66,-1.0,0.996012,10915717.68,0.995459,0.995811,0.995911,25176.48,0.996162,21018.74,0.996048,14.410019,25462.0,0.0,0.0
5039493,1.0,462.0,530.0,941325.89,-1.0,0.996363,10908949.84,0.995459,0.995861,0.995911,16652.16,0.996614,6149.78,0.996425,13.209581,25463.0,0.0,0.0
5039693,1.0,462.0,540.0,896737.81,-1.0,0.996012,10933632.53,0.995459,0.995861,0.996012,17050.36,0.996564,133701.38,0.996074,17.230510,25464.0,0.0,0.0


## calculate median volumns

do this after handling NaNs in far and near price

In [45]:
%%time 

# Creating an output dataframe for the results:-
median_vol = \
pd.DataFrame(columns = ['overall_medvol', "first5min_medvol", "last5min_medvol"], 
                          index = range(0, 200,1)
            );

# Creating the overall median volumes- this is from public work:-
median_vol['overall_medvol'] = \
df[['stock_id', "bid_size", "ask_size"]].\
groupby("stock_id")[["bid_size", "ask_size"]].\
median().\
sum(axis=1).\
values.flatten();

# Creating median volume with near and far price information:-
median_vol['last5min_medvol'] = \
df[['stock_id', "bid_size", "ask_size", "far_price", "near_price"]].\
dropna().\
groupby('stock_id')[["bid_size", "ask_size"]].\
median().sum(axis=1).values.flatten();

# # Creating median volume without near and far price information:-
median_vol['first5min_medvol'] = \
df.loc[(df['syn_far_price']==1) | (df['syn_near_price']==1), 
            ['stock_id', "bid_size", "ask_size"]
           ].\
groupby('stock_id')[["bid_size", "ask_size"]].\
median().sum(axis=1).values.flatten();


# PrintColor(f"\n---> Median stock price information-  sample = 10 rows\n");
# display(median_vol.sample(10).style.format(formatter = '{:,.2f}'));

# Writing the dataframe to a csv file for later usage:-
# median_vol.to_csv(f"MedianVolV{version_nb}.csv")

collect();
median_vol

CPU times: user 843 ms, sys: 0 ns, total: 843 ms
Wall time: 841 ms


,overall_medvol,first5min_medvol,last5min_medvol
0,42739.16,33806.880,42739.16
1,25548.50,19033.920,25548.50
2,26228.10,19536.000,26228.10
3,41667.00,37667.500,41667.00
4,34014.58,30904.545,34014.58
...,...,...,...
195,51941.55,38331.170,51941.55
196,42476.95,30245.125,42476.95
197,30070.04,23667.870,30070.04
198,304739.25,234244.150,304739.25


## Now drop all other NaN rows

### TODO is this wise?

In [46]:
df.dropna(inplace=True)

# Parallel Triplet Imbalance Calculation

In [48]:
from numba import njit, prange

# @njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = 0  #TODO changed from np.NaN to 0
                # print(f"ERROR max_val-min_val={max_val-min_val} mid_val={mid_val}")
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features


In [49]:
%%time
#TODO take out
sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
    triplet_feature = calculate_triplet_imbalance_numba(c, df)
    df[triplet_feature.columns] = triplet_feature.values
df.head()

CPU times: user 57.6 s, sys: 16 ms, total: 57.7 s
Wall time: 57.7 s


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,syn_near_price,syn_far_price,ask_price_bid_price_wap_imb2,ask_price_bid_price_reference_price_imb2,ask_price_wap_reference_price_imb2,bid_price_wap_reference_price_imb2,matched_size_bid_size_ask_size_imb2,matched_size_bid_size_imbalance_size_imb2,matched_size_ask_size_imbalance_size_imb2,bid_size_ask_size_imbalance_size_imb2
0,0.0,0.0,0.0,3180602.69,1.0,0.999812,13380276.64,0.0,0.0,0.999812,...,1.0,1.0,0.138298,-1.927541e+12,0.138298,-1.693353e+12,255.368402,3.269177,3.215423,59.816770
1,1.0,0.0,0.0,166603.91,-1.0,0.999896,1642214.25,0.0,0.0,0.999896,...,1.0,1.0,6.346154,-6.881500e+12,6.346154,9.367487e+11,93.345872,9.032273,10.107002,8.404237
2,2.0,0.0,0.0,302879.87,-1.0,0.999561,1819368.03,0.0,0.0,0.999403,...,1.0,1.0,0.499162,4.664557e+00,0.678815,2.778481e+00,93.951375,5.724241,5.341913,13.972041
3,3.0,0.0,0.0,11917682.27,-1.0,1.000171,18389745.62,0.0,0.0,0.999999,...,1.0,1.0,214.000000,2.500000e-01,0.251462,1.710000e+02,37.571704,0.543170,0.565807,23.995112
4,4.0,0.0,0.0,447549.96,-1.0,0.999532,17860614.95,0.0,0.0,0.999394,...,1.0,1.0,0.026403,3.507246e+00,0.034188,3.391304e+00,1111.684024,40.395505,38.945308,26.855187


In [50]:
df.isnull().sum()

stock_id                                     0
date_id                                      0
seconds_in_bucket                            0
imbalance_size                               0
imbalance_buy_sell_flag                      0
reference_price                              0
matched_size                                 0
far_price                                    0
near_price                                   0
bid_price                                    0
bid_size                                     0
ask_price                                    0
ask_size                                     0
wap                                          0
target                                       0
time_id                                      0
syn_near_price                               0
syn_far_price                                0
ask_price_bid_price_wap_imb2                 0
ask_price_bid_price_reference_price_imb2     0
ask_price_wap_reference_price_imb2           0
bid_price_wap

In [23]:
# [col for col in df1.columns if col not in df.columns]

['ask_price_bid_price_wap_imb2',
 'ask_price_bid_price_reference_price_imb2',
 'ask_price_wap_reference_price_imb2',
 'bid_price_wap_reference_price_imb2',
 'matched_size_bid_size_ask_size_imb2',
 'matched_size_bid_size_imbalance_size_imb2',
 'matched_size_ask_size_imbalance_size_imb2',
 'bid_size_ask_size_imbalance_size_imb2']

# Save and Reload

In [27]:
# %%time
# # !pip install pyarrow

# df.to_feather("./tmp.feather")
# df = pd.read_feather("./tmp.feather")
# df.shape

CPU times: user 367 ms, sys: 127 ms, total: 494 ms
Wall time: 132 ms


(659945, 18)

In [51]:
%%time
df.to_parquet("./tmpdfgb.parquet")
df.shape

CPU times: user 5.34 s, sys: 1.14 s, total: 6.49 s
Wall time: 4.2 s


(5237760, 26)

In [52]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,syn_near_price,syn_far_price,ask_price_bid_price_wap_imb2,ask_price_bid_price_reference_price_imb2,ask_price_wap_reference_price_imb2,bid_price_wap_reference_price_imb2,matched_size_bid_size_ask_size_imb2,matched_size_bid_size_imbalance_size_imb2,matched_size_ask_size_imbalance_size_imb2,bid_size_ask_size_imbalance_size_imb2
0,0.0,0.0,0.0,3180602.69,1.0,0.999812,13380276.64,0.0,0.0,0.999812,...,1.0,1.0,0.138298,-1.927541e+12,0.138298,-1.693353e+12,255.368402,3.269177,3.215423,59.816770
1,1.0,0.0,0.0,166603.91,-1.0,0.999896,1642214.25,0.0,0.0,0.999896,...,1.0,1.0,6.346154,-6.881500e+12,6.346154,9.367487e+11,93.345872,9.032273,10.107002,8.404237
2,2.0,0.0,0.0,302879.87,-1.0,0.999561,1819368.03,0.0,0.0,0.999403,...,1.0,1.0,0.499162,4.664557e+00,0.678815,2.778481e+00,93.951375,5.724241,5.341913,13.972041
3,3.0,0.0,0.0,11917682.27,-1.0,1.000171,18389745.62,0.0,0.0,0.999999,...,1.0,1.0,214.000000,2.500000e-01,0.251462,1.710000e+02,37.571704,0.543170,0.565807,23.995112
4,4.0,0.0,0.0,447549.96,-1.0,0.999532,17860614.95,0.0,0.0,0.999394,...,1.0,1.0,0.026403,3.507246e+00,0.034188,3.391304e+00,1111.684024,40.395505,38.945308,26.855187


# Feature Generation Functions 

In [53]:
df=pd.read_parquet("./tmpdfgb.parquet")
df.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'syn_near_price',
       'syn_far_price', 'ask_price_bid_price_wap_imb2',
       'ask_price_bid_price_reference_price_imb2',
       'ask_price_wap_reference_price_imb2',
       'bid_price_wap_reference_price_imb2',
       'matched_size_bid_size_ask_size_imb2',
       'matched_size_bid_size_imbalance_size_imb2',
       'matched_size_ask_size_imbalance_size_imb2',
       'bid_size_ask_size_imbalance_size_imb2'],
      dtype='object')

In [54]:
def imbalance_features(df):
    # Define lists of price and size-related column names
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")
        df[f"{c[0]}_{c[1]}_imb"].fillna(0,inplace=True)  #if both are 0 then imbalance is 0
        

    # for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
    #     triplet_feature = calculate_triplet_imbalance_numba(c, df)
    #     df[triplet_feature.columns] = triplet_feature.values
   
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    
    # Calculate various statistical aggregation features
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        

    for col in ['matched_size', 'imbalance_size', 'reference_price', 'imbalance_buy_sell_flag']:
        for window in [1, 2, 3, 6]:
            df[f"{col}_shift_{window}"] = df.groupby('stock_id')[col].shift(window)
            df[f"{col}_shift_{window}"].fillna(0,inplace=True)
            df[f"{col}_ret_{window}"] = df.groupby('stock_id')[col].pct_change(window)
            df[f"{col}_ret_{window}"].fillna(0,inplace=True)
            
    # # Calculate diff features for specific columns
    # for col in ['ask_price', 'bid_price', 'ask_size', 'bid_size', 'market_urgency', 'imbalance_momentum', 'size_imbalance']:
    #     for window in [1, 2, 3, 10]:
    #         df[f"{col}_diff_{window}"] = df.groupby("stock_id")[col].diff(window)

    return df.replace([np.inf, -np.inf], 0)

def other_features(df):
    df["dow"] = df["date_id"] % 5  # Day of the week
    # df["seconds"] = df["seconds_in_bucket"] % 60  
    # df["minute"] = df["seconds_in_bucket"] // 60  
    # for key, value in global_stock_id_feats.items():
    #     df[f"global_{key}"] = df["stock_id"].map(value.to_dict())

    return df

def generate_all_features(df):
    # Select relevant columns for feature generation
    cols = [c for c in df.columns if c not in ["row_id", "time_id"]]
    df = df[cols]
    
    # Generate imbalance features
    df = imbalance_features(df)
    df = other_features(df)
    gc.collect()  
    feature_name = [i for i in df.columns if i not in ["row_id", "time_id"]]
    
    return df[feature_name]


In [55]:
%%time
df = generate_all_features(df)

CPU times: user 17.4 s, sys: 4.18 s, total: 21.6 s
Wall time: 21.6 s


In [56]:
df1=df.iloc[1200:,:]
df1.fillna(0,inplace=True)

In [57]:
%%time
df1.to_parquet("./tmpalldfgb.parquet")
df=pd.read_parquet("./tmpalldfgb.parquet")

CPU times: user 18.4 s, sys: 7.05 s, total: 25.5 s
Wall time: 17 s


In [58]:
# remove syn generated near_price and far price rows
df=df[df.syn_near_price==0]
bs.remove_syn_columns(df)